# The Transaction class
## Main components of a transaction
A transaction constitutes the fundamental component of a quantitative trading architecture. A transaction can be defined as an object with the following attributes:
1. Date
2. Ticker symbol
3. Number of shares
4. Total transaction amount
5. Transaction type
6. Description

## Additional implications
The following considerations will be discussed later:
1. The per share amount in a transaction can be derived from the total transaction amount and the number of shares
2. A ticker symbol does not always have to be associated with a transaction. This behavior is present in a bank transfer of cash into or out of a brokerage account

## Defining the transaction class
Assume first that the initialized transaction contains no actual information. This is a defensive coding practice that motivates the use of the None as the default value for all attributes. The expected data type of each of the class attributes is included as a comment, and because the datetime.date class is needed, the datetime library is imported

In [14]:
import datetime
class Transaction:
    """
    Represents a bank transfer, buy, dividends, fee, or sell
    """
    def __init__(self, date=None, symbol=None, num_shares=None,
                 total_amount=None, transact_type=None, description=None):
        self.date = date  # Should be datetime.date class
        self.symbol = symbol  # Should be string
        self.num_shares = num_shares  # Should be integer
        self.total_amount = total_amount  # Should be float
        self.transact_type = transact_type  # Should be string
        self.description = description  # Should be string

# Several demonstrative transactions
Consider the following theoretical transactions:
1. The purchase of 20 shares of Apple stock at a total price of \\$4000, placed on October 10th of 2018. Note that a price of \\$200 per share can be calculated from this information.
2. A bank transfer of \\$500 into a brokerage account on June 9th of 2019. Note that this transaction does not include a ticker symbol and as such the None keyword is used to intialize the bank transfer transaction object.

In [15]:
t1 = Transaction(datetime.date(2018, 10, 12), 'AAPL', 20, 4000, 'Buy', "Apple market buy")
t2 = Transaction(datetime.date(2019, 6, 9), None, 500, "Bank transfer", "Transfer to brokerage account")

In [16]:
print(t1.date, t1.symbol, t1.num_shares, t1.total_amount, t1.transact_type, t1.description)
print(t2.date, t2.symbol, t2.num_shares, t2.total_amount, t2.transact_type, t2.description)

2018-10-12 AAPL 20 4000 Buy Apple market buy
2019-06-09 None 500 Bank transfer Transfer to brokerage account None


# TODO:
1. Use a pandas dataframe(?) to show the transactions in a table
2. Write the test code based upon the demonstrative transactions. The first steps should be simply that the data is returned. Add in some test cases for extra nones, etc.